In [2]:
import tensorflow as tf

In [3]:
def read_and_decode(filename):
    filename_queue = tf.train.string_input_producer([filename])# create a queue
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)#return file_name and file
    features = tf.parse_single_example(serialized_example,
                                       features={
                                           'image':tf.FixedLenFeature([],tf.string),
                                           'label':tf.FixedLenFeature([],tf.int64),
                                       })#return image and label
    # 获取图片数据
    image = tf.decode_raw(features['image'],tf.uint8)
    image = tf.reshape(image,[40,500])  #reshape image to 40*40
    image = tf.cast(image, tf.float32) / 255.0
    label = tf.cast(features['label'],tf.int64) #throw label tensor
    return image, label

# 初始化权值
def weight_variable(shape,name):
    initial = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial,name=name)

# 初始化偏置
def bias_variable(shape,name):
    initial = tf.constant(0.1,shape=shape)
    return tf.Variable(initial,name=name)

# 卷积层
def conv2d(x,W):
    # x input tensor of shape [batch,in_height,in_width,in_channels]
    # W filter/kernel tensor of shape [filter_height,filter_width,in_channels,out_channels]
    # strides[0] = strides[3] = 1,strides[1]代表x方向步长,strides[2]代表y方向步长
    # padding: 'SAME' or 'VALID'
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

# 池化层
def max_pool_2x2(x):
    # ksize [1,x,y,1]
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')


    # 定义两个placeholder
    x = tf.placeholder(tf.float32,[None,40,50],name='x_input')
    y = tf.placeholder(tf.float32,[None],name='y_input')
    X = tf.reshape(x,[25,40,50,1],name='x_image')
    Y = tf.one_hot(indices=tf.cast(y,tf.int32),depth=13)
  
        
with tf.name_scope('Conv1'):
    # 初始化第一个卷积层的权值和偏置
    with tf.name_scope('W_conv1'):
        W_conv1 = weight_variable([5,5,1,32],name='W_conv1') # 5*5采样窗口,32个卷积核从1个平面抽取特征
    with tf.name_scope('b_conv1'):
        b_conv1 = bias_variable([32],name='b_conv1') # 每一个卷积核只需要一个偏置值

    # 把x_image和权值向量进行卷积,再加上偏置值,然后应用于relu激活函数
    with tf.name_scope('conv2d_1'):
        conv2d_1 = conv2d(X,W_conv1) + b_conv1
    with tf.name_scope('relu'):
        h_conv1 = tf.nn.relu(conv2d_1)
    with tf.name_scope('h_pool1'): 
        h_pool1 = max_pool_2x2(h_conv1) # 进行max-pooling

with tf.name_scope('Conv2'):
    # 初始化第二个卷积层的权值和偏置
    with tf.name_scope('W_conv2'):
        W_conv2 = weight_variable([5,5,32,64],name='W_conv2') # 5*5采样窗口,64个卷积核从32个平面抽取特征
    with tf.name_scope('b_conv2'):
        b_conv2 = bias_variable([64],name='b_conv1') # 每一个卷积核只需要一个偏置值
    
    # 把h_pool1和权值向量进行卷积,再加上偏置值,然后应用于relu激活函数
    with tf.name_scope('conv2d_2'):
        conv2d_2 = conv2d(h_pool1,W_conv2) + b_conv2
    with tf.name_scope('relu'):
        h_conv2 = tf.nn.relu(conv2d_2)
    with tf.name_scope('h_pool2'): 
        h_pool2 = max_pool_2x2(h_conv2) # 进行max-pooling

# 40*40的图片第一次卷积后还是40*40,第一次池化后变为20*20
# 第二次卷积后为20*20,第二次池化后变为10*10
# 经过上面的操作后得到64张10*10的平面

with tf.name_scope('fc1'):
    # 初始化第一个全连接层的权值
    with tf.name_scope('W_fc1'):
        W_fc1 = weight_variable([10*13*64,1024],name='W_fc1') # 上一层有7*7*64个神经元,全连接层有1024个神经元
    with tf.name_scope('b_fc1'):
        b_fc1 = bias_variable([1024],name='b_fc1') # 1024个节点

    with tf.name_scope('h_pool2_flat'):
        # 把池化层2的输出扁平化为1维
        h_pool2_flat = tf.reshape(h_pool2,[-1,10*13*64])

    with tf.name_scope('wx_plus_b1'):
        wx_plus_b1 = tf.matmul(h_pool2_flat,W_fc1) + b_fc1
    with tf.name_scope('relu'):
        # 求第一个全连接层的输出
        h_fc1 = tf.nn.relu(wx_plus_b1)
    with tf.name_scope('keep_prob'):
        # keep_prob用来表示神经元的输出概率
        keep_prob = tf.placeholder(tf.float32)
    with tf.name_scope('h_fc1_drop'):
        h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)

# 初始化第二个全连接层
with tf.name_scope('fc2'):
    with tf.name_scope('W_fc2'):
        W_fc2 = weight_variable([1024,13],name='W_fc2')
    with tf.name_scope('b_fc2'):
        b_fc2 = bias_variable([13],name='b_fc2')
    with tf.name_scope('xw_plus_b2'):
        wx_plus_b2 = tf.matmul(h_fc1,W_fc2) + b_fc2
    with tf.name_scope('softmax'):
        # 计算输出
        prediction = tf.nn.softmax(wx_plus_b2)

# 交叉熵代价函数     
with tf.name_scope('cross_entropy'):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y,logits=prediction))
    tf.summary.scalar('cross_entropy',cross_entropy)

# 使用AdamOptimizer进行优化
with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# 求准确率
with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        # 结果存放在一个布尔列表中
        correct_prediction = tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
    with tf.name_scope('accuracy'):
        # 求准确率
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
        tf.summary.scalar('accuracy',accuracy)

# 合并所有的summary
merged = tf.summary.merge_all()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    train_writer = tf.summary.FileWriter('logs/train',sess.graph)
    image,label = read_and_decode("F:/cut/image_train_1-of-1.tfrecord")
    image_batch,label_batch = tf.train.shuffle_batch(
        [image,label],batch_size=25,capacity=50000,min_after_dequeue=10000
    )
    
    for i in range(10001):
        # 获取一个批次的数据和标签
        batch_xs,batch_ys = sess.run([image_batch,label_batch])
        # 训练模型
        sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys,keep_prob:1.0})
        # 记录训练集计算的参数
        summary = sess.run(merged,feed_dict={x:batch_xs,y:batch_ys,keep_prob:1.0})
        train_writer.add_summary(summary,i)
        train_acc = sess.run(accuracy,feed_dict={x:batch_xs,y:batch_ys,keep_prob:1.0})
        print("Iter " + str(i) + ", Training Accuracy = " + str(train_acc))